# Overview
* Create an array of 3 parquet files to load
    * Create a simple CSV, then read it in as a CSV
* Create an outer loop of all the parquet files
* For each parquet, create a delta table in databricks using Apache Spark
    * https://www.projectpro.io/recipes/create-delta-table-with-existing-data-databricks
```
//reading source file and writing to destination path 
// Skip this since we are reading in a parquet 
spark.read.option("inferschema",true).option("header",true).csv("/FileStore/tables/sample_emp_data.txt")

// find a way to write a local delta table instead of the Databricks FileStore?
write.format("delta").mode("overwrite").save("/FileStore/tables/delta_train/")
//Below we are listing the data in destination path 
display(dbutils.fs.ls("/FileStore/tables/delta_train/"))
```


In [ ]:
import dask.dataframe as dd
import pandas as pd
import findspark
from pyspark import DeltaTable
from pyspark import spark


findspark.init()


# Small Parquet example
filenames_list = ["smallTable.csv", "largeTable.csv"]
parquetList = []

for n, f in enumerate(filenames_list):
    df = dd.read_csv(f, compression="gzip", blocksize=None)
    df = df.repartition(partition_size="100MB")
    df.write.format("parquet").saveAsTable("parquetTable_" + n)
    parq = df.to_parquet(parquet_file, write_index=False)
    parquetList.append(parq)


for n, p in enumerate(parquetList):
    df = pd.read_parquet(path, engine='auto', columns=None, storage_options=None, use_nullable_dtypes=False, **kwargs)
    df.write.format("delta").saveAsTable("deltaTable_" + n)
    DeltaTable.isDeltaTable(spark, "deltaTable_" + n) # True
    spark.table("deltaTable_" + n).show()










